# The (7,4) Hamming Code

As a preliminary, we define a modulo 2 function.

In [1]:
mod2(x) = mod(x,2)

mod2 (generic function with 1 method)

For encoding, we need the $\mathbf{G}$ matrix (Eq. 1.28 in MacKay, 2003)

In [2]:
G = [1 0 0 0 1 0 1; 0 1 0 0 1 1 0; 0 0 1 0 1 1 1; 0 0 0 1 0 1 1]

4×7 Array{Int64,2}:
 1  0  0  0  1  0  1
 0  1  0  0  1  1  0
 0  0  1  0  1  1  1
 0  0  0  1  0  1  1

Now we simply post-multiply $\mathbf{G}$ to the source message $\mathbf{s}$ to get the codeword $\mathbf{t}$. In order to get a binary codeword, we need to take the result modulo 2.

In [3]:
s = [1 0 1 1]

1×4 Array{Int64,2}:
 1  0  1  1

In [4]:
t = s*G .|> mod2

1×7 Array{Int64,2}:
 1  0  1  1  0  0  1

The binary symmetric channel flips any transmitted bit with probability $f$. We use this property to generate a noise vector $\mathbf{n}$.

In [5]:
using Random

In [6]:
f = 0.1

0.1

In [7]:
n = permutedims(rand(7).<f)

1×7 BitArray{2}:
 1  0  0  1  0  0  0

The received vector $\mathbf{r}$ is the sum (modulo 2) of the transmitted codeword $\mathbf{t}$ and the noise $\mathbf{n}$.

In [8]:
r = t + n .|> mod2

1×7 Array{Int64,2}:
 0  0  1  0  0  0  1

For decoding, we need the *parity-check matrix* $\mathbf{H}$.

In [9]:
H = [1 1 1 0 1 0 0; 0 1 1 1 0 1 0; 1 0 1 1 0 0 1]

3×7 Array{Int64,2}:
 1  1  1  0  1  0  0
 0  1  1  1  0  1  0
 1  0  1  1  0  0  1

This enables us to calculate the *syndrome* $\mathbf{z}$ of the received vector $\mathbf{r}$.

In [10]:
z = r*H' .|> mod2

1×3 Array{Int64,2}:
 1  1  0

Now we need a function that maps the syndrome $\mathbf{z}$ onto the most likely noise vector $\mathbf{\hat{n}}$ (most likely for small flip rates $f$, that is).

In [11]:
function nhat(z)
    if z == [0 0 0]
        n = [0 0 0 0 0 0 0]
    elseif z == [0 0 1]
        n = [0 0 0 0 0 0 1]
    elseif z == [0 1 0]
        n = [0 0 0 0 0 1 0]
    elseif z == [0 1 1]
        n = [0 0 0 1 0 0 0]
    elseif z == [1 0 0]
        n = [0 0 0 0 1 0 0]
    elseif z == [1 0 1]
        n = [1 0 0 0 0 0 0]
    elseif z == [1 1 0]
        n = [0 1 0 0 0 0 0]
    elseif z == [1 1 1]
        n = [0 0 1 0 0 0 0]
    else
        error("Input is not a valid syndrome")
    end
    
    return(n)
end

nhat (generic function with 1 method)

In [12]:
nhat(z)

1×7 Array{Int64,2}:
 0  1  0  0  0  0  0

Our best guess $\mathbf{\hat{t}}$ for the transmitted vector $\mathbf{t}$ is the sum (modulo 2) of the received vector $\mathbf{r}$ and $\mathbf{\hat{n}}$. If at most one bit was flipped (i.e., $\mathbf{n}$ contained at most one non-zero element), then our guess will be correct. However, if more than one bit was flipped during transmission, our guess will be wrong.

In [13]:
that = r + nhat(z) .|> mod2

1×7 Array{Int64,2}:
 0  1  1  0  0  0  1

Finally, our best guess $\mathbf{\hat{s}}$ for the source vector is the first four bits of $\mathbf{\hat{t}}$.

In [14]:
shat = that[:,1:4]

1×4 Array{Int64,2}:
 0  1  1  0